In [21]:
# Required Libraries
import pandas as pd
import os

In [22]:
# Paths to the data

# SINAN DataSUS CSV files path (modify to match your file path)
sinan_path = os.path.expanduser('~/Desktop/DataSUS-Chikungunya-ML/source/csv/')

# Cleaned CSV files path (modify to match your file path)
cleaned_path = os.path.expanduser('~/Desktop/DataSUS-Chikungunya-ML/datasets/')

In [23]:
"""
Load the Chikungunya data from the CSV files
The CSV files are named CHIKBRYY.csv, where YY is the last two digits of the year
The files are stored in the ~/Downloads/dbc2csv/source/csv/ directory
The columns in the CSV files are not consistent across all years
We want to identify the columns that are common to all files
"""

# List of last two digits of years for which we have CSV files (2018 to 2024)
start_year = 18
end_year = 25

# Number of years to use for testing
test_years = 2

assert start_year < end_year, "Start year must be less than end year"
years = range(start_year, end_year + 1)

assert len(years) > test_years, "At least {test_years + 1}  years of data are required."

# Dictionary to store the columns for each file
file_columns = {}

# Loop through each year, build the filename, and read the CSV
for year in years:
    file_name = f'{sinan_path}CHIKBR{str(year)}.csv'
    try:
        df = pd.read_csv(file_name, low_memory=False)
        # Save the set of columns for this file
        file_columns[file_name] = set(df.columns)
    except Exception as e:
        print(f"Error loading {file_name}: {e}")

# Ensure we have loaded at least one file before proceeding
assert file_columns, "No files were loaded. Please check your file paths."

# Find common columns: the intersection of columns across all files
common_columns = set.intersection(*file_columns.values())
assert common_columns, "No common columns found. Please check your file paths."

print("\nColumns common to all files:")
print(common_columns)

# Compute the union of all columns (all columns that appear in any file)
all_columns = set.union(*file_columns.values())

# For columns that are not common, print which files have them and which don't.
print("\nColumns that are not common among all files:")
for col in all_columns - common_columns:
    # Extract base name (e.g., CHIKBR21) from each file path
    files_with = [os.path.splitext(os.path.basename(fname))[0] 
                    for fname, cols in file_columns.items() if col in cols]
    files_without = [os.path.splitext(os.path.basename(fname))[0] 
                        for fname, cols in file_columns.items() if col not in cols]
    print(f"Column '{col}' is present in files: {files_with} and missing in files: {files_without}")



Columns common to all files:
{'ALRM_ABDOM', 'GRAV_MELEN', 'ARTRALGIA', 'DT_ENCERRA', 'ID_RG_RESI', 'ALRM_HEMAT', 'ARTRITE', 'FEBRE', 'SEM_PRI', 'UF', 'CRITERIO', 'ID_MUNICIP', 'CS_GESTANT', 'VOMITO', 'GENGIVO', 'DOR_COSTAS', 'RES_CHIKS1', 'MIALGIA', 'NU_IDADE_N', 'TPAUTOCTO', 'RESUL_SORO', 'ID_REGIONA', 'SG_UF_NOT', 'ALRM_LIQ', 'NDUPLIC_N', 'CEFALEIA', 'MUNICIPIO', 'CON_FHD', 'RESUL_PCR_', 'SG_UF', 'RES_CHIKS2', 'COMPLICA', 'RENAL', 'DIABETES', 'GRAV_MIOC', 'CS_RACA', 'COMUNINF', 'ALRM_LETAR', 'GRAV_ENCH', 'GRAV_HIPOT', 'ALRM_VOM', 'DT_CHIK_S1', 'LACO_N', 'GRAV_HEMAT', 'HEMATURA', 'HEPATOPAT', 'DT_CHIK_S2', 'DT_NS1', 'GRAV_SANG', 'LACO', 'DT_INTERNA', 'CLASSI_FIN', 'AUTO_IMUNE', 'ID_MN_RESI', 'DT_ALRM', 'DT_PRNT', 'GRAV_INSUF', 'HIPERTENSA', 'DT_NOTIFIC', 'NU_ANO', 'ID_OCUPA_N', 'DT_PCR', 'COPAISINF', 'DT_GRAV', 'GRAV_ORGAO', 'SOROTIPO', 'IMUNOH_N', 'HOSPITALIZ', 'DT_VIRAL', 'PLASMATICO', 'EXANTEMA', 'GRAV_METRO', 'ALRM_SANG', 'DOR_RETRO', 'RESUL_VI_N', 'MANI_HEMOR', 'HISTOPA_N', 'DOE

In [24]:
"""
Remove columns that are not common to all files
Create a new DataFrame with only the common columns
Concatenate all the DataFrames without the last {test_years} years into a single DataFrame called X_train
Concatenate the DataFrames from the last {test_years} years into a single DataFrame called X_test
"""

# Load the train and test data for each year, keeping only the common columns
X_train = []
X_test = []

for year in years:
    file_name = f'{sinan_path}CHIKBR{str(year)}.csv'
    try:
        df = pd.read_csv(file_name, usecols=common_columns, low_memory=False)
        # Drop the "Unnamed: 0" column if present
        if "Unnamed: 0" in df.columns:
            df = df.drop("Unnamed: 0", axis=1)
        if year < years[-test_years]:
            X_train.append(df)
        else:
            X_test.append(df)
    except Exception as e:
        print(f"Error loading {file_name}: {e}")

# Concatenate all the DataFrames into a single train and test DataFrame
X_train = pd.concat(X_train, ignore_index=True)
X_test = pd.concat(X_test, ignore_index=True)

# Display the shape of the concatenated DataFrame
print("\nShape of the concatenated X_train DataFrame: ", X_train.shape)
print("Shape of the concatenated X_test DataFrame: ", X_test.shape)



Shape of the concatenated X_train DataFrame:  (1050480, 116)
Shape of the concatenated X_test DataFrame:  (449729, 116)


In [25]:
"""
Filter the data to include only the rows where the pacient was hospitalized ("HOSPITALIZ" column is equal to 1 or to 1.0)
"""

print("\nUnique values in the 'HOSPITALIZ' column:")
print(X_train['HOSPITALIZ'].value_counts())

# Filter the data to include only the rows where the patient was either hospitalized or not (remove ignored values)
X_train = X_train[(X_train['HOSPITALIZ'] == 1) | (X_train['HOSPITALIZ'] == 1.0) | (X_train['HOSPITALIZ'] == 2) | (X_train['HOSPITALIZ'] == 2.0)]
X_test = X_test[(X_test['HOSPITALIZ'] == 1) | (X_test['HOSPITALIZ'] == 1.0) | (X_test['HOSPITALIZ'] == 2) | (X_test['HOSPITALIZ'] == 2.0)]

# Display the shape of the filtered DataFrame
print("\nShape of the filtered X_train DataFrame: ", X_train.shape)
print("Shape of the filtered X_test DataFrame: ", X_test.shape)


Unique values in the 'HOSPITALIZ' column:
HOSPITALIZ
2.0    537813
2       73033
1.0     28859
9.0     22044
1        3629
9        2885
           1
Ø           1
J           1
ï           1
Name: count, dtype: int64

Shape of the filtered X_train DataFrame:  (566672, 116)
Shape of the filtered X_test DataFrame:  (311808, 116)


In [26]:
# Remove the 'HOSPITALIZ' column from X_train and X_test and use it as the target label
y_train = X_train.pop("HOSPITALIZ")
y_test = X_test.pop("HOSPITALIZ")

# Map the target: 1 (positive) becomes 1, 2 (negative) becomes 0
y_train = y_train.map({1: 1, 2: 0})
y_test = y_test.map({1: 1, 2: 0})

# Print the value counts for the target variable
print("\nValues for 'HOSPITALIZ' in y_train:")
print(y_train.value_counts(dropna=True))

print("\nValues for 'HOSPITALIZ' in y_test:")
print(y_test.value_counts(dropna=True))


Values for 'HOSPITALIZ' in y_train:
HOSPITALIZ
0    537813
1     28859
Name: count, dtype: int64

Values for 'HOSPITALIZ' in y_test:
HOSPITALIZ
0    297865
1     13943
Name: count, dtype: int64


In [27]:
# Remove columns where all values are the same in X_train and X_test (constant columns)
constant_columns = [col for col in X_train.columns if X_train[col].nunique() == 1]

# Drop these constant columns from both X_train and X_test
X_train = X_train.drop(columns=constant_columns)
X_test = X_test.drop(columns=constant_columns, errors='ignore')

print("\nDropped constant columns (constant values):", constant_columns)


Dropped constant columns (constant values): ['TP_NOT', 'ID_AGRAVO', 'ID_PAIS', 'SOROTIPO', 'HISTOPA_N', 'TP_SISTEMA', 'NDUPLIC_N']


In [28]:
"""
Identify and remove columns with more than missing_values_threshold missing values in X_train and X_test
"""

missing_values_threshold = 0.20
assert 0.0 <= missing_values_threshold <= 1.0, "missing_values_threshold must be between 0 and 1"

# Compute the percentage of missing values in each column of X_train
missing_values_train = X_train.isnull().mean()
missing_values_test = X_test.isnull().mean()
missing_values_mean = (missing_values_train + missing_values_test) / 2

# Print the percentage of missing values in each column of X_train
print("\nPercentage of missing values in X_train:")
print(missing_values_train)

# Print the percentage of missing values in each column of X_test
print("\nPercentage of missing values in X_test:")
print(missing_values_test)



Percentage of missing values in X_train:
DT_NOTIFIC    0.0
SEM_NOT       0.0
NU_ANO        0.0
SG_UF_NOT     0.0
ID_MUNICIP    0.0
             ... 
PLASMATICO    1.0
EVIDENCIA     1.0
PLAQ_MENOR    1.0
CON_FHD       1.0
COMPLICA      1.0
Length: 108, dtype: float64

Percentage of missing values in X_test:
DT_NOTIFIC    0.0
SEM_NOT       0.0
NU_ANO        0.0
SG_UF_NOT     0.0
ID_MUNICIP    0.0
             ... 
PLASMATICO    1.0
EVIDENCIA     1.0
PLAQ_MENOR    1.0
CON_FHD       1.0
COMPLICA      1.0
Length: 108, dtype: float64


In [29]:
cols_to_drop = missing_values_train[missing_values_mean > missing_values_threshold].index

X_train = X_train.drop(columns=cols_to_drop)
X_test = X_test.drop(columns=cols_to_drop)

print(f"\nDropped columns (more than {missing_values_threshold * 100}% missing):")
print(list(cols_to_drop))


Dropped columns (more than 20.0% missing):
['ID_OCUPA_N', 'DT_CHIK_S1', 'DT_CHIK_S2', 'DT_PRNT', 'RES_CHIKS1', 'RES_CHIKS2', 'RESUL_PRNT', 'DT_SORO', 'RESUL_SORO', 'DT_NS1', 'RESUL_NS1', 'DT_VIRAL', 'RESUL_VI_N', 'DT_PCR', 'RESUL_PCR_', 'IMUNOH_N', 'DT_INTERNA', 'UF', 'MUNICIPIO', 'TPAUTOCTO', 'COUFINF', 'COPAISINF', 'COMUNINF', 'DOENCA_TRA', 'CLINC_CHIK', 'DT_OBITO', 'ALRM_HIPOT', 'ALRM_PLAQ', 'ALRM_VOM', 'ALRM_SANG', 'ALRM_HEMAT', 'ALRM_ABDOM', 'ALRM_LETAR', 'ALRM_HEPAT', 'ALRM_LIQ', 'DT_ALRM', 'GRAV_PULSO', 'GRAV_CONV', 'GRAV_ENCH', 'GRAV_INSUF', 'GRAV_TAQUI', 'GRAV_EXTRE', 'GRAV_HIPOT', 'GRAV_HEMAT', 'GRAV_MELEN', 'GRAV_METRO', 'GRAV_SANG', 'GRAV_AST', 'GRAV_MIOC', 'GRAV_CONSC', 'GRAV_ORGAO', 'DT_GRAV', 'MANI_HEMOR', 'EPISTAXE', 'GENGIVO', 'METRO', 'PETEQUIAS', 'HEMATURA', 'SANGRAM', 'LACO_N', 'PLASMATICO', 'EVIDENCIA', 'PLAQ_MENOR', 'CON_FHD', 'COMPLICA']


In [30]:
# Display the shape of the filtered DataFrame
print("\nShape of the filtered X_train DataFrame: ", X_train.shape)
print("Shape of the filtered X_test DataFrame: ", X_test.shape)


Shape of the filtered X_train DataFrame:  (566672, 43)
Shape of the filtered X_test DataFrame:  (311808, 43)


In [31]:
"""
Save the filtered data to a new CSV file in the cleaned_path directory
"""

# Save the filtered data to a new CSV file in the cleaned_path directory
X_train.to_csv(f'{cleaned_path}X_train.csv', index=False)
y_train.to_csv(f'{cleaned_path}y_train.csv', index=False)

X_test.to_csv(f'{cleaned_path}X_test.csv', index=False)
y_test.to_csv(f'{cleaned_path}y_test.csv', index=False)